In [ ]:
pip install huffman

In [ ]:
import collections
from collections import deque
import huffman

#NodeTree

In [ ]:
class NodeTree(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def children(self):
        return (self.left, self.right)

    def nodes(self):
        return (self.left, self.right)

    def __str__(self):
        return '%s_%s' % (self.left, self.right)

#Huffman Code tree

In [ ]:
def huffman_code_tree(node, left=True, binString=''):
    if type(node) is str:
        return {node: binString}
    (l, r) = node.children()
    d = dict()
    d.update(huffman_code_tree(l, True, binString + '0'))
    d.update(huffman_code_tree(r, False, binString + '1'))
    return d

#Huffman Codes

In [ ]:
def huffman_codes_0(string):
	freq = {}
	for c in string:
		if c in freq:
			freq[c] += 1
		else:
			freq[c] = 1

	freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)

	nodes = freq

	while len(nodes) > 1:
		(key1, c1) = nodes[-1]
		(key2, c2) = nodes[-2]
		nodes = nodes[:-2]
		node = NodeTree(key1, key2)
		nodes.append((node, c1 + c2))

		nodes = sorted(nodes, key=lambda x: x[1], reverse=True)
	huffmanCode = huffman_code_tree(nodes[0][0])

	print(' Char | HF code | Code length')
	print('--------------------------------')
	for (char, frequency) in freq:
		print(' %-4r |%8s |%5s' % (char, huffmanCode[char], len(huffmanCode[char])))
		codebook[char] = huffmanCode[char]

	return codebook

##version_2

In [ ]:
def huffman_codes_1(string):
    codebook = huffman.codebook(collections.Counter(string).items())
    print(codebook)
    return codebook

#canonical code Tree

In [ ]:
def can_code_tree(codebook):  
    codebook = dict(sorted(dict(sorted(codebook.items())).items(), key=lambda x: len(x[1])))
    can_code=[]
    i=0
    for key, value in codebook.items() :
        length = len(value)
        if i==0:
            can_code.append('0'*length)
        else:
            bin_inc = int(can_code[i-1], base=2) + 1
            bin_str = str("{:b}".format(bin_inc))
            msb_zero = len(can_code[i-1])-len(bin_str)
            lsb_zero = length-len(can_code[i-1])
            bin_str = ('0'*msb_zero) + bin_str    
            bin_str = bin_str + ('0'*abs(lsb_zero))
            can_code.append(bin_str)
        i = i+1
    codebook_Can = {list(codebook.keys())[i]: can_code[i] for i in range(len(can_code))} 
    return codebook_Can

# T_bar(C)

In [ ]:
def T_bar(C, codebook_Can):
    r_max = len(C)
    for r in range(1, r_max+1):
        C_temp = []
        C_temp = (C[0:r])
        # print(C_temp,r)
        for key, value in codebook_Can.items() :
            if C_temp == list(value) :
                z = key
                return (z, r)
    return -1

# Encoding

In [ ]:
def E_t(S, N, t, T_book) :
    X=[-1]*N*t
    stack_T1= deque()

    for i in range(1, N+1):
        C = T_book[S[i-1]]
        L = len(C)
        start = (i-1)*t
        end = i*t
        if L <= t :
            end_temp = (i-1)*t + L
            X[start : end_temp] = C[ : L]
            X[end_temp : end] = [stack_T1.pop() for _ in range(t-L) if len(stack_T1)>0]
        else :
            X[start : end] = C[ : t]
            for i in range(t, L) :
                stack_T1.append(C[i])
            
    I = 0
    if len(stack_T1) == 0:
        return (X,0)
    if len(stack_T1) != 0 :            
        for i in range(0, N*t) :
          if len(stack_T1) == 0 :
            break 
          if X[i] == -1 :
            X[i] = stack_T1.pop()
            I = i
    while len(stack_T1) != 0 :
      X.append(stack_T1.pop())

    I = int(I/t) + 1
    return (X, I)

# Decoding

##Decoding1.1

In [ ]:
# def F(s, e, T) :
#     S=[-1]*e
#     i = e
#     for i in range(e,s,-1):
#         start = (i-1)*t  #indexing
#         end = i*t
#         C = X[start : end]
#         while T_bar(C) == -1:
#             top = T.pop()
#             C = C + [top]
#         (S[i-1], r) = T_bar(C)
#         for i in range(r,t):
#             T.append(C[i-1])
#     return (S, T)

In [ ]:
def F(s, e, T, codebook_Can):
  S=[-1]*e
  i = e
  while i!=s :
    start = (i-1)*t  #indexing
    end = i*t
    C = X[start : end]
    if len(T) == 0 :
        (S[i-1], r) = T_bar(C, codebook_Can)
    else :
        while T_bar(C, codebook_Can) == -1:
          top = T.pop()
          C = C + [top]
        (S[i-1], r) = T_bar(C, codebook_Can)
    if r!=t and r<t:
      temp=C[r:t]
      for j in range(len(temp)):
        T.append(temp[j])
    i=i-1
  return S[s:e]

## Decoding1.2

In [ ]:
def Decode(X, M, I, N, t, codebook_Can):
  S=[-1]*N
  start=0
  end=N*t
  X1=X[start:end]
  T1=deque()
  if (M>end):
    start=N*t
    end=M
    X2=X[start:end]
    for i in range(0,len(X2)):
      T1.append(X2[i])
  if I==0:
    S=F(0, N, T1, codebook_Can)
  else:
    T2=deque()
    #T2.append('-1')
    S[0:I]=F(0, I, T2, codebook_Can)
    while len(T2)!=0:
      temp=T2.pop()
      T1.append(temp[0])
    S[I:N]=F(I, N, T1, codebook_Can)
  return S

# Retrieval

In [ ]:
def R(X, t, N, k):
    M = len(X)
    if k > N:
        return ('-', k, [])
        # return ('-1', k, X[N*t : M])
    else:
        C = X[(k-1)*t : k*t]
    while True:
        if T_bar(C, codebook_Can) != -1:
            (v, r) = T_bar(C, codebook_Can)
            return (v, k, C[r: t])
        elif k+1 <= N:
            (V, K, B) = R(X,t,N,k+1)
        else:
            return ('-',N+1,[])
        C = C + B
        k = K

## retrieval_bar

In [ ]:
def Ra_bar(X, t, N, k) :
    if k > N :
        return print("error")
    start1 = (k-1)*t #indexing
    end1 = k*t
    C =  X[start1 : end1]
 
    if T_bar(C) != -1 :
        (v, r) = T_bar(C)
        print(r)
        return v
    else :
        j = T[S[k-1]] - t #indexing
        m = 0
        if j + m > 0 :
            while m >= 0 :
                k += 1
                m = m + T[S[k-1]] - t #indexing
                
            start2 = k*t + m -1 -1 #indexing
            end2 = k*t
            # 16: C ← (C|X(kt + m − 1 : kt)
            C = (C +  X[start2 : end2])
 
    (v, r) = T_bar(C)
    return v

# Test case Inputs

In [ ]:
# key:value = test_case : block_size

text_file ={
    "uditupadhayay":3,
    "minion":2,
    "error":2,
    "project":2,
    "datastructure":3,
    "alphabet":3
}

text = list(text_file.keys())[-2] #give index number inside "[]""
t = text_file[text]
N = len(text)
S = [char for char in text]

In [ ]:
codebook = {}
codebook = huffman_codes_0(text)
codebook_Can = can_code_tree(codebook)

print(codebook_Can)

 Char | HF code | Code length
--------------------------------
 't'  |      01 |    2
 'a'  |      00 |    2
 'r'  |     101 |    3
 'u'  |     100 |    3
 'd'  |    1101 |    4
 's'  |    1100 |    4
 'c'  |    1111 |    4
 'e'  |    1110 |    4
{'a': '00', 't': '01', 'r': '100', 'u': '101', 'c': '1100', 'd': '1101', 'e': '1110', 's': '1111'}


In [ ]:
(X, I) = E_t(S, N, t, codebook_Can)

print(X,I)

['1', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', -1, '1', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1', '1'] 3


In [ ]:
# X = ['1','1','0','1','0','0','1','0','0','0','1','0','0','1','1','1','0','1','1','0','1','1','0','1','1','1','0']
# I = 1
# print(X)

In [ ]:
M = len(X)
N = len(text)

decode_text_list = Decode(X, M, I, N, t, codebook_Can)
decode_text = "".join(decode_text_list)

print(decode_text)

datastructure


In [ ]:
print("Input text : \"" + text + "\". Output text : \"" + decode_text)
print(text==decode_text)

Input text : "datastructure". Output text : "datastructure
True


#### Retrieve Symbol at random index

In [ ]:
X = ['1', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1', '1']
t = 3
text = 'datastructure'
N = len(text)

codebook = {}
codebook = huffman_codes_0(text)
codebook_Can = can_code_tree(codebook)

print(codebook_Can)

 Char | HF code | Code length
--------------------------------
 't'  |      01 |    2
 'a'  |      00 |    2
 'r'  |     101 |    3
 'u'  |     100 |    3
 'd'  |    1101 |    4
 's'  |    1100 |    4
 'c'  |    1111 |    4
 'e'  |    1110 |    4
{'a': '00', 't': '01', 'r': '100', 'u': '101', 'c': '1100', 'd': '1101', 'e': '1110', 's': '1111'}


In [ ]:
# retrieve symbols from 5 to N-1
out = []
for i in range(5,N):
  v,_,_ = R(X,t,N,i)
  out.append(v)
print(out)

['s', 't', 'r', 'u', 'c', 't', 'u', 'r']
